In [29]:
# Import packages
import math

In [30]:
# Define physical properties
length = 8  # m
delta_h = 12  # in.
thickness = 0.5  # mm
D_i = 0.8  # cm
D_o = 1.2  # cm
minor_losses = 25
V_wort = 6  # gal
T_0_wort = 200  # F
T_f_wort = 70  # F
T_city = 50  # F
P_water = 80  # psi

In [31]:
# Define material properties
k_tube = 1  # units
h_water = 1  # units
mu_water = 8.9e-4  # Pa*s
P_atm = 101325  # Pa
rho_water = 1000  # kg/m^3
g = 9.81  # m/s^2

In [32]:
# Import pump curve
# pump curve fit to polynomial in form c1 + c2x + c3x^2 + c4x^3

In [33]:
# conversion functions
def in_to_m(measurement):
    return 0.0254 * measurement

def mm_to_m(measurement):
    return measurement / 1000

def cm_to_m(measurement):
    return measurement / 100

def gal_to_m3(measurement):
    return measurement * 0.00378541

def f_to_c(measurement):
    return (measurement - 32) * 5 / 9

def psi_to_pa(measurement):
    return measurement * 6894.76

In [34]:
# Convert to SI units
delta_h = in_to_m(delta_h)
thickness = mm_to_m(thickness)
D_i = cm_to_m(D_i)
D_o = cm_to_m(D_o)
V_wort = gal_to_m3(V_wort)
T_0_wort = f_to_c(T_0_wort)
T_f_wort = f_to_c(T_f_wort)
T_city = f_to_c(T_city)
P_water = psi_to_pa(P_water)

# Derived values
D_h_out = D_o - (D_i + 2 * thickness)

In [35]:
# Head Loss Equation
def head_loss(f, L, D, K, v, g):
    return (f * (L / D) + K) * (v ** 2) / (2 * g)

In [36]:
# Reynolds Number Equation
def reynolds(rho, v, D, mu):
    return rho * v * D / mu

In [66]:
# Friction Factor Equations
def friction_factor(Re):
    return (0.79 * math.log(Re) - 1.64) ** -2

def ff_check(dp, D, rho, v):
    # dp is pressure drop in Pa
    return (2 * dp * D) / ( rho * v ** 2)

In [38]:
# Velocity equation
def velocity(dp, dz, h_in, h_l_total, rho, g):
    # positive dp is pressure drop, positive dz is drop in height
    p_term = dp / (rho * g)
    z_term = dz
    h_term = h_in - h_l_total
    return (2 * g * (p_term + z_term + h_term)) ** (1/2)

In [69]:
# Calculate the flow rate of the annulus
f = 0.02  # starting guess
h_l = head_loss(f, length, D_h_out, 0, 0.1, g)
print(f"Head Loss: {h_l} m")
delta_P = P_water - P_atm

# Start by assuming no losses
converged = False
v_out = velocity(delta_P / 1000, 0, 0, h_l, rho_water, g)
print(f"Velocity: {v_out} m/s")
i = 0
while not converged:
    print(f"\nIteration {i}")
    Re = reynolds(rho_water, v_out, D_h_out, mu_water)
    print(f"Reynolds Number: {Re}")
    f = friction_factor(Re)
    print(f"Friction Factor: {f}")
    h_l = head_loss(f, length, D_h_out, 0, v_out, g)
    print(f"Head Loss: {h_l} m")
    v_2 = velocity(delta_P / 1000, 0, 0, h_l, rho_water, g)
    print(f"Velocity: {v_2} m/s")
    i += 1
    dv = abs(v_out - v_2)
    v_out = v_2
    if dv <= 0.05 * v_2:
        converged = True
print(f"Completed in {i} iterations")
print(f"Flow Velocity of Annulus: {v_out} m/s")
m_dot = rho_water * v_out * math.pi * ((D_o / 2) ** 2 - ((D_i + 2 * thickness) / 2) ** 2)
print(f"Flow Rate of Annulus")

Head Loss: 0.027183146449201504 m
Velocity: 0.6059523633642058 m/s

Iteration 0
Reynolds Number: 2042.5360562838391
Friction Factor: 0.0520938412868271
Head Loss: 2.599758932749658 m
Velocity: (4.3344002318266035e-16+7.078612763850576j) m/s


TypeError: '<=' not supported between instances of 'float' and 'complex'